In [120]:
import requests
import re 
import time
from bs4 import BeautifulSoup 
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from random import randint
import selenium.webdriver

In [121]:
city_url = 'https://raw.githubusercontent.com/ucsb-dreamlab/dianpingBusinessScrape/master/City_URL.csv'
city = pd.read_csv(city_url, index_col= 0)
city.head(10)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
5,安顺,www.dianping.com/anshun/ch75/g34311
6,阿里,www.dianping.com/ali/ch75/g34311
7,安康,www.dianping.com/ankang/ch75/g34311
8,安国市,www.dianping.com/anguo/ch75/g34311
9,安新县,www.dianping.com/anxin/ch75/g34311


Filter this dataset by city levels. Now it has 411 results, and most cities in this filtered dataset are tier 1-3 city. 

In [122]:
filtered_city = city[-(
                city['city_name'].str.endswith('市')| 
                city['city_name'].str.endswith('县')|
                city['city_name'].str.endswith('镇')| 
                city['city_name'].str.endswith('旗')|
                city['city_name'].str.endswith('村')| 
                city['city_name'].str.endswith('区'))].reset_index(drop=True)
display(filtered_city)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
...,...,...
406,诸暨,www.dianping.com/zhuji/ch75/g34311
407,章丘,www.dianping.com/zhangqiu/ch75/g34311
408,周庄,www.dianping.com/zhouzhuang/ch75/g34311
409,朱家尖,www.dianping.com/zhujiajian/ch75/g34311


In [123]:
from selenium.webdriver import ChromeOptions

In [124]:
!pip install mitmproxy

1. https://stackoverflow.com/questions/33225947/can-a-website-detect-when-you-are-using-selenium-with-chromedriver
2. https://stackoverflow.com/questions/55501524/how-does-recaptcha-3-know-im-using-selenium-chromedriver/55502835#55502835
3. https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec



In [116]:
def getHtmlContent(url): 
 UA = UserAgent(verify_ssl = False)  
 #Retrieve cookies with selenium
 options = ChromeOptions()
 options.add_argument("--disable-blink-features=AutomationControlled")
 driver = selenium.webdriver.Chrome(options= options)
 
 #driver_option.add_argument("--headless")
 print(driver.execute_script("return navigator.userAgent;"))
 driver.get(url)
 all_cookies = driver.get_cookies()
 cookies_dict = {}
 for cookies in all_cookies:
   cookies_dict[cookies['name']] = cookies['value']
 
 #Set User Agent in request headers
 
 headers = {
    "User-Agent": UA.random
    }
 #Request HTML with session
 try:
   s = requests.Session()
   #r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
   r = s.get(url, headers = headers, cookies= cookies_dict)
   r.raise_for_status()
   if(r.encoding != "UTF-8"):
      r.encoding = r.apparent_encoding
   return r.text
 except:
   return "Request Error"

In [110]:
URL =  "http://"+ filtered_city['URL'][30] # add "https" to avoid schema error 
URL

'http://www.dianping.com/changchun/ch75/g34311'

In [117]:
HTML = getHtmlContent(URL)

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36


In [118]:
soup = BeautifulSoup(HTML, "html.parser")
print(soup)

<html>
<head>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="icon" type="image/x-icon"/>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="shortcut icon" type="image/x-icon"/>
<title>长春绘本馆机构_课程_价格_排名-大众点评网</title>
<meta content="长春绘本馆,长春绘本馆机构,长春绘本馆课程,长春绘本馆价格,长春绘本馆机构排名,大众点评网" name="Keywords"/>
<meta content="大众点评网学习培训频道为您找到最新最全的长春绘本馆机构。点击查看更多关于长春绘本馆机构课程、价格、评价、排名、电话、地址等信息。" name="Description"/>
<meta content="format=html5; url=https://m.dianping.com/changchun/ch75/g34311" http-equiv="mobile-agent"/>
<link href="https://m.dianping.com/changchun/ch75/g34311" media="only screen and (max-width: 640px)" rel="alternate"/>
<meta charset="utf-8">
<script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '70',                   
            'cityChName': '长春',                 
            'cityEnName': 'changchun',               
            'pageType': 'search',         

In [113]:
SubURL = []
LibName = []
for div in soup.find_all(class_ = "tit"):
    for a in div.find_all('a', href = True):
        SubURL.append(a.get('href'))
        LibName.append(a.getText().strip())

//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [114]:
city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
shop_lib = city_lib[city_lib['URL'].str.contains("shop")]
shop_lib

,Library_Name,URL
0,吉的堡成长中心(虹馆校),https://www.dianping.com/shop/HaeWsJ0w5WL7QCk1
3,七田全脑潜能开发训练中心(桂林路校区),https://www.dianping.com/shop/l2iZPtapMFdaKJR6
6,七田全脑潜能开发训练中心(净月校区),https://www.dianping.com/shop/ETEgfeIFdAeU44xA
9,爱育幼童儿童脑潜能开发(新天地购物公园亚泰大街店),https://www.dianping.com/shop/l267ntjVOMsX60oo
12,七田全脑潜能开发训练中心(上海路分校),https://www.dianping.com/shop/FNPcwF48oczZ4vrF
15,吉的堡成长中心(长春中新校区),https://www.dianping.com/shop/k9fNrbjIbrWPWGCq
18,司蒂姆儿童手创俱乐部(高新店),https://www.dianping.com/shop/H8oVFAf1R2msAmh1
20,七田家哆啦育童儿童成长中心,https://www.dianping.com/shop/H3w0cfzTV1QS8yNl
22,BOOKplay书戏教育(南湖分店),https://www.dianping.com/shop/l1Fp5bA6ApIQpcfh
23,安妮花阅读馆,https://www.dianping.com/shop/GaN0H8K4Zfo9ya8X


In [119]:
subpage_soup = []
for i in range(0,len(shop_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = shop_lib['URL'][i]), "html.parser"))
    time.sleep(randint(1,2))

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36


KeyError: 1

In [27]:
subpage_soup[7]

<!DOCTYPE html>

<html>
<head>
<link href="http://www.dianping.com/shop/G8ZSkMu9B7e8uQqw" rel="canonical"/>
<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="http://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}if(document.referrer.indexOf("baidu.com")>0){if(window.location.search==""){e=e+"?msource=baidusearch"}else{e=e+"&msource=baidusearch"}}else if(document.referrer.indexOf("sogou.com")>0){if(window.location.search==""){e=e+"?msource=sogousearch"}else{e=e+"&msource=sogousearch"}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>
<title>喜阅绘本馆课程_价格_简介_怎么样-安庆学习培训-大众点评网</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="pc" name="applicable-device"/>
<meta content="喜阅绘本馆,喜阅绘本馆课

In [28]:
AddressList = []
HourList = []
PhoneNumList = []


In [29]:
for soup in subpage_soup:
    for div in soup.find_all(class_ = "address"):
            AddressList.append(div.find_all(text=True, recursive=False)[1].strip())

In [30]:
AddressList

['工农街68号-116号',
 '文津路与西三路交叉路口向西南约150米',
 '华茂C8栋二层207-210室',
 '蓝鼎商业街西区148号',
 '吾悦华府二期(为百姓大药房楼上)11S2101室',
 '回祥路130号（东方虹城小区大门口）',
 '人民路70号',
 '皖江大道迎江世纪城绿地启航社3幢1室2-3层',
 '汇峰广场二楼',
 '吾悦广场金街三楼3082（宜城渡步行街)',
 '皖西南大道115号',
 '后围墙与东围墙街交叉口西140米',
 '联合街037号',
 '西后街与龙眠西路交叉口西南50米']

In [31]:

for soup in subpage_soup:
    hour_tag = soup.find(class_ = "mod shop-info")
    if hour_tag is not None:
       for li in hour_tag.find_all('li'):
          str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
          HourList.append(str) 
    else:
          HourList.append(np.NaN)

In [17]:
     #for li in div.find_all('li'):
         #   try:
           #     HourList.append(li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")) 
            #except IndexError:
              #  HourList.append(np.NaN)
                   

In [32]:
HourList

['周二至周日 09:30-20:00',
 nan,
 '周二至周日 10:30-21:00',
 nan,
 '周一至周日 10:00-22:00',
 '周一至周日 09:30-20:00',
 nan,
 '周一至周日 10:00-22:00',
 nan,
 '周一至周日 10:00-22:00',
 nan,
 nan,
 nan,
 '周一至周日 09:00-21:00']

In [33]:
for soup in subpage_soup: 
    phone_tag = soup.find(class_ = "phone")
    if phone_tag is not None:
        phonenum = phone_tag.find('span')
        PhoneNumList.append(phonenum['data-phone'])
    else: 
        PhoneNumList.append(np.NaN)

In [34]:
PhoneNumList

['13866049075',
 '15609626576',
 '18905561200',
 '13917865342',
 '13225562850',
 '18726151009',
 nan,
 '13170061015',
 nan,
 '0556-5268540',
 nan,
 '18956990189',
 '13855611597',
 '13966987786']

In [35]:
city_lib['Address'] = AddressList
city_lib['Business_Hour'] = HourList
city_lib['Phone_Number'] = PhoneNumList
city_lib

,Library_Name,URL,Address,Business_Hour,Phone_Number
0,老约翰绘本馆(壕埂街店),https://www.dianping.com/shop/EsBovbrmyg1vwAzC,工农街68号-116号,周二至周日 09:30-20:00,13866049075
1,时光工作室,https://www.dianping.com/shop/iIFyKwy9eGUQbtFn,文津路与西三路交叉路口向西南约150米,NaN,15609626576
2,童声童色中英文亲子绘本馆(华茂店),https://www.dianping.com/shop/G27b4iGNXv38tXhw,华茂C8栋二层207-210室,周二至周日 10:30-21:00,18905561200
3,维多利亚全英文绘本馆,https://www.dianping.com/shop/H5KwHtJ6DypQm0fW,蓝鼎商业街西区148号,NaN,13917865342
4,喜山美阅绘本馆(吾悦华府店),https://www.dianping.com/shop/H7UvmjQzGBUYztnN,吾悦华府二期(为百姓大药房楼上)11S2101室,周一至周日 10:00-22:00,13225562850
5,十月绘本馆(菱建小区店),https://www.dianping.com/shop/H1wgdkiXPFhwTpzc,回祥路130号（东方虹城小区大门口）,周一至周日 09:30-20:00,18726151009
6,墨念文创,https://www.dianping.com/shop/l7TRfgIGlx5ciTpY,人民路70号,NaN,NaN
7,喜阅绘本馆,https://www.dianping.com/shop/G8ZSkMu9B7e8uQqw,皖江大道迎江世纪城绿地启航社3幢1室2-3层,周一至周日 10:00-22:00,13170061015
8,国王的礼物(汇峰店),https://www.dianping.com/shop/l1W07Cl06lQ1OyTl,汇峰广场二楼,NaN,NaN
9,优丫漫绘本美学馆(安庆新城吾悦广场店),https://www.dianping.com/shop/H39YMzNFBzVEA23g,吾悦广场金街三楼3082（宜城渡步行街),周一至周日 10:00-22:00,0556-5268540


In [39]:
#Summarized the above code into a function
def parse_info_citylib(URL):
    subpage_soup = []
    for i in range(0,6): #Need to change this part
        subpage_soup.append(BeautifulSoup(getHtmlContent(url = URL), "html.parser"))
        time.sleep(3)

    AddressList = []
    HourList = []
    PhoneNumList = []

    for soup in subpage_soup:
        for div in soup.find_all(class_ = "address"):
            AddressList.append(div.find_all(text=True, recursive=False)[1].strip())
    
    for soup in subpage_soup:    
        hour_tag = soup.find(class_ = "mod shop-info")
        if hour_tag is not None:
            for li in hour_tag.find_all('li'):
                str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                HourList.append(str) 
            else:
                HourList.append(np.NaN)
    
    for soup in subpage_soup:
        phone_tag = soup.find(class_ = "phone")
        if phone_tag is not None:
            phonenum = phone_tag.find('span')
            PhoneNumList.append(phonenum['data-phone'])
        else: 
            PhoneNumList.append(np.NaN)
            
    return AddressList, HourList, PhoneNumList

In [47]:
Second_URL="http://"+ filtered_city['URL'][30]
lib_info = parse_info_citylib(Second_URL)

In [50]:
lib_info[2]

[nan, nan, nan, nan, nan, nan]